In [34]:
import numpy as np
import pandas as pd
import time

In [24]:
# Starting variables

# From-to Distance and Flow matrices start with Receiving and end with Shipping
# and all functions are written accordingly since Receiving cannot have inflow
# and Shipping cannot have outflow.

distance = np.array([[0, 12, 24, 6, 18, 36, 24,30, 36], 
                    [0, 0, 6, 24, 12, 30, 18, 24, 30], 
                    [0, 30, 0, 24, 36, 30, 18, 12, 30], 
                    [0, 24, 36, 0, 6, 24, 12, 18, 24],
                    [0, 24, 36, 18, 0, 24, 12, 6, 24], 
                    [0, 18, 30, 12, 24, 0, 30, 36, 42], 
                    [0, 18, 30, 12, 24, 6, 0, 36, 6], 
                    [0, 24, 36, 18, 30, 12, 36, 0, 12],
                    [0, 0, 0, 0, 0, 0, 0, 0, 0]])

flows = np.array([[0, 8, 0, 0, 0, 0, 12, 0, 0], 
                 [0, 0, 0, 0, 4, 4, 0, 4, 0], 
                 [0, 0, 0, 0, 0, 0, 6, 0, 0], 
                 [0, 0, 0, 0, 0, 0, 0, 0, 0], 
                 [0, 0, 0, 0, 0, 0, 0, 4, 5], 
                 [0, 4, 6, 0, 5, 0, 0, 1, 0], 
                 [0, 0, 0, 0, 0, 12, 0, 0, 6], 
                 [0, 0, 0, 0, 0, 0, 0, 0, 9],
                 [0, 0, 0, 0, 0, 0, 0, 0, 0]])

machines = ["Receiving", "VTC2", "UMC", "Buffer", "VMC", "SHP", "VTC1", "HMC", "Shipping"]

In [25]:
# Changes rows and columns of given matrix given two indexes
def changed_matrix(matrix, i1, i2):
    result = matrix.copy()
    result[[i1,i2]] = matrix[[i2,i1]] # row change
    result[:, [i1, i2]] = result[:, [i2, i1]] # column change
    
    return result

In [26]:
# Takes input of flow as ixi matrix, machine as list, and fixed location index list 
# The algorithm does not take into account of items of list.
# Returns the best flow that has lowest cost.

def steepest_descent(flow:list, machine:list, fixed_loc:list):
    min_change = (0, 0, 0) # tuple
    machine_list = machine
    min_cost = np.multiply(flow, distance).sum()
    
    # O(nlogn)
    # Starts from 1 (Receiving not included) and ends at 8 (Shipping not included)
    # Calculates total cost of new flow for every number excluding fixed locations.
    for r1 in range(1, 8):
        if r1 in fixed_loc: continue # Buffer (fixed)
        for r2 in range(r1+1, 8):
            if r2 in fixed_loc: continue # Buffer (fixed)
            new_flow = changed_matrix(flow.copy(), r1, r2)
            changed = np.multiply(new_flow, distance).sum()
            
            if min_cost - changed > min_change[2]:
                min_change = (r1, r2, min_cost - changed)
                min_cost = changed
    
    # End of iterations
    if min_change[2] > 0:
        # Location of the best swap
        change_1 = min_change[0]
        change_2 = min_change[1]
        # New flow
        new_flow = changed_matrix(flow.copy(), change_1, change_2)

        # New Machine
        new_machine = machine.copy()
        new_machine[change_1], new_machine[change_2] = new_machine[change_2], new_machine[change_1]

        # Total score
        total_score = np.multiply(new_flow, distance).sum()
        return new_flow, total_score, new_machine, change_1, change_2
    
    return 0, 0, 0, 0, 0

In [27]:
print_all = True # If true, prints all iterations
count = 0 # iteration count
start = time.time() # start time

# First iteration including buffer
opt_flow, improvement, opt_machine, ch1, ch2 = steepest_descent(flows, machines, [])

print("Including Buffer")
# Main loop
while improvement != 0:
    count += 1
    if print_all:
        db = pd.DataFrame(opt_flow, columns=opt_machine, index=opt_machine)
        display(db)
        print(f"Total: {improvement}, changed: {opt_machine[ch1]} and {opt_machine[ch2]}")
    else:
        lst_opt_flow, lst_improvement, lst_opt_machine, lst_ch1, lst_ch2 = opt_flow, improvement, opt_machine, ch1, ch2
        
    opt_flow, improvement, opt_machine, ch1, ch2 = steepest_descent(opt_flow, opt_machine, [])
    
if 1 - print_all:
    db = pd.DataFrame(lst_opt_flow, columns=lst_opt_machine, index=lst_opt_machine)
    display(db)
    print(f"Total: {lst_improvement}, changed: {lst_opt_machine[lst_ch1]} and {lst_opt_machine[lst_ch2]}")

print(f"time: {time.time()-start}, iteration count: {count}")

Including Buffer


,Receiving,Buffer,UMC,VTC2,VMC,SHP,VTC1,HMC,Shipping
Receiving,0,0,0,8,0,0,12,0,0
Buffer,0,0,0,0,0,0,0,0,0
UMC,0,0,0,0,0,0,6,0,0
VTC2,0,0,0,0,4,4,0,4,0
VMC,0,0,0,0,0,0,0,4,5
SHP,0,0,6,4,5,0,0,1,0
VTC1,0,0,0,0,0,12,0,0,6
HMC,0,0,0,0,0,0,0,0,9
Shipping,0,0,0,0,0,0,0,0,0


Total: 1380, changed: Buffer and VTC2


,Receiving,UMC,Buffer,VTC2,VMC,SHP,VTC1,HMC,Shipping
Receiving,0,0,0,8,0,0,12,0,0
UMC,0,0,0,0,0,0,6,0,0
Buffer,0,0,0,0,0,0,0,0,0
VTC2,0,0,0,0,4,4,0,4,0
VMC,0,0,0,0,0,0,0,4,5
SHP,0,6,0,4,5,0,0,1,0
VTC1,0,0,0,0,0,12,0,0,6
HMC,0,0,0,0,0,0,0,0,9
Shipping,0,0,0,0,0,0,0,0,0


Total: 1308, changed: UMC and Buffer
time: 0.01793813705444336, iteration count: 2


In [28]:
print_all = True # If true, prints all iterations
count = 0 # iteration count
start = time.time() # start time

# First iteration including buffer
opt_flow, improvement, opt_machine, ch1, ch2 = steepest_descent(flows, machines, [3])

print("Excluding Buffer")

# Main loop
while improvement != 0:
    count += 1
    if print_all:
        db = pd.DataFrame(opt_flow, columns=opt_machine, index=opt_machine)
        display(db)
        print(f"Total: {improvement}, changed: {opt_machine[ch1]} and {opt_machine[ch2]}")
    else:
        lst_opt_flow, lst_improvement, lst_opt_machine, lst_ch1, lst_ch2 = opt_flow, improvement, opt_machine, ch1, ch2
        
    opt_flow, improvement, opt_machine, ch1, ch2 = steepest_descent(opt_flow, opt_machine, [3])
    
if 1 - print_all:
    db = pd.DataFrame(lst_opt_flow, columns=lst_opt_machine, index=lst_opt_machine)
    display(db)
    print(f"Total: {lst_improvement}, changed: {lst_opt_machine[lst_ch1]} and {lst_opt_machine[lst_ch2]}")

print(f"time: {time.time()-start}, iteration count: {count}")

Excluding Buffer


,Receiving,VTC2,VMC,Buffer,UMC,SHP,VTC1,HMC,Shipping
Receiving,0,8,0,0,0,0,12,0,0
VTC2,0,0,4,0,0,4,0,4,0
VMC,0,0,0,0,0,0,0,4,5
Buffer,0,0,0,0,0,0,0,0,0
UMC,0,0,0,0,0,0,6,0,0
SHP,0,4,5,0,6,0,0,1,0
VTC1,0,0,0,0,0,12,0,0,6
HMC,0,0,0,0,0,0,0,0,9
Shipping,0,0,0,0,0,0,0,0,0


Total: 1512, changed: VMC and UMC
time: 0.010181903839111328, iteration count: 1


In [29]:
# A descent algorithm that works the same way as steepest_descent function
# the difference of two is that the descent algorithm stops when finds a better swap than
# the initial flow.

def descent(flow:list, machine:list, fixed_loc:list):
    machine_list = machine
    min_cost = np.multiply(flow, distance).sum()
    
    for r1 in range(1, 8):
        if r1 in fixed_loc: continue
        for r2 in range(r1+1, 8):
            if r2 in fixed_loc: continue
            new_flow = changed_matrix(flow.copy(), r1, r2)
            changed = np.multiply(new_flow, distance).sum()
            
            if min_cost - changed > 0:
                new_flow = changed_matrix(flow.copy(), r1, r2)
                new_machine = machine.copy()
                new_machine[r1], new_machine[r2] = new_machine[r2], new_machine[r1]
                
                return new_flow, new_machine, np.multiply(new_flow, distance).sum()
            
    return 0, 0, 0

In [30]:
# Takes the same inputs as steepest_descent function
# The difference is in calculation of the best swap, which does not change
# the flow matrix in every for loop, only calculates the swapped lanes' cost improvement.

# Works slower but have less memory complexity.

def improved_steepest_descent(flow:list, machine:list, fixed_loc:list):
    min_change = (0, 0, 0) # tuple
    machine_list = machine
    
    # O(nlogn)
    for r1 in range(1, 8):
        if r1 in fixed_loc: continue
        for r2 in range(r1+1, 8):
            if r2 in fixed_loc: continue
            first_cost = np.multiply(flow[[r1,r2]], distance[[r1,r2]]).sum() + np.multiply(flow[:, [r1,r2]], distance[:, [r1,r2]]).sum() - (flow[r2][r1]*distance[r2][r1]) - (flow[r1][r2]*distance[r1][r2])
            changed = np.multiply(flow[[r1,r2]], distance[[r2,r1]]).sum() + np.multiply(flow[:, [r1,r2]], distance[:, [r2,r1]]).sum() + (flow[r2][r1]*distance[r1][r2]) + (flow[r1][r2]*distance[r2][r1])
            
            if changed - first_cost < min_change[2]:
                min_change = (r1, r2, changed - first_cost)
    
    # End of iterations
    if min_change[2] < 0:
        change_1 = min_change[0]
        change_2 = min_change[1]
        # New flow
        new_flow = changed_matrix(flow.copy(), change_1, change_2)

        # New Machine
        new_machine = machine.copy()
        new_machine[change_1], new_machine[change_2] = new_machine[change_2], new_machine[change_1]

        # Total score
        total_score = np.multiply(new_flow, distance).sum()
        return new_flow, total_score, new_machine, change_1, change_2
    
    return 0, 0, 0, 0, 0

In [31]:
print_all = True # If true, prints all iterations
start = time.time() # start time
count = 0 # iteration count

# First iteration with buffer included
imp_opt_flow, imp_improvement, imp_opt_machine, imp_ch1, imp_ch2 = improved_steepest_descent(flows, machines, [])

print("Buffer Included")

# Main loop
while imp_improvement != 0:
    count += 1
    if print_all:
        db = pd.DataFrame(imp_opt_flow, columns=imp_opt_machine, index=imp_opt_machine)
        display(db)
        print(f"Total: {imp_improvement}, changed: {imp_opt_machine[imp_ch1]} and {imp_opt_machine[imp_ch2]}")
    else:
        last_opt_flow, last_improvement, last_opt_machine, last_imp_ch1, last_imp_ch2 = imp_opt_flow, imp_improvement, imp_opt_machine, imp_ch1, imp_ch2

    imp_opt_flow, imp_improvement, imp_opt_machine, imp_ch1, imp_ch2 = improved_steepest_descent(imp_opt_flow, imp_opt_machine, [])

if 1 - print_all:
    db = pd.DataFrame(last_opt_flow, columns=last_opt_machine, index=last_opt_machine)
    display(db)
    print(f"Total: {last_improvement}, changed: {last_opt_machine[last_imp_ch1]} and {last_opt_machine[last_imp_ch2]}")
        
print(f"time: {time.time()-start}, iteration count: {count}")

Buffer Included


,Receiving,Buffer,UMC,VTC2,VMC,SHP,VTC1,HMC,Shipping
Receiving,0,0,0,8,0,0,12,0,0
Buffer,0,0,0,0,0,0,0,0,0
UMC,0,0,0,0,0,0,6,0,0
VTC2,0,0,0,0,4,4,0,4,0
VMC,0,0,0,0,0,0,0,4,5
SHP,0,0,6,4,5,0,0,1,0
VTC1,0,0,0,0,0,12,0,0,6
HMC,0,0,0,0,0,0,0,0,9
Shipping,0,0,0,0,0,0,0,0,0


Total: 1380, changed: Buffer and VTC2


,Receiving,UMC,Buffer,VTC2,VMC,SHP,VTC1,HMC,Shipping
Receiving,0,0,0,8,0,0,12,0,0
UMC,0,0,0,0,0,0,6,0,0
Buffer,0,0,0,0,0,0,0,0,0
VTC2,0,0,0,0,4,4,0,4,0
VMC,0,0,0,0,0,0,0,4,5
SHP,0,6,0,4,5,0,0,1,0
VTC1,0,0,0,0,0,12,0,0,6
HMC,0,0,0,0,0,0,0,0,9
Shipping,0,0,0,0,0,0,0,0,0


Total: 1308, changed: UMC and Buffer
time: 0.019079208374023438, iteration count: 2


In [32]:
print_all = True # If true, prints all iterations
start = time.time() # start time
count = 0 # iteration count

# First iteration excluding buffer
imp_opt_flow, imp_improvement, imp_opt_machine, imp_ch1, imp_ch2 = improved_steepest_descent(flows, machines, [3])

print("Buffer Excluded")

# Main loop
while imp_improvement != 0:
    count += 1
    if print_all:
        db = pd.DataFrame(imp_opt_flow, columns=imp_opt_machine, index=imp_opt_machine)
        display(db)
        print(f"Total: {imp_improvement}, changed: {imp_opt_machine[imp_ch1]} and {imp_opt_machine[imp_ch2]}")
    else:
        last_opt_flow, last_improvement, last_opt_machine, last_imp_ch1, last_imp_ch2 = imp_opt_flow, imp_improvement, imp_opt_machine, imp_ch1, imp_ch2

    imp_opt_flow, imp_improvement, imp_opt_machine, imp_ch1, imp_ch2 = improved_steepest_descent(imp_opt_flow, imp_opt_machine, [3])

if 1 - print_all:
    db = pd.DataFrame(last_opt_flow, columns=last_opt_machine, index=last_opt_machine)
    display(db)
    print(f"Total: {last_improvement}, changed: {last_opt_machine[last_imp_ch1]} and {last_opt_machine[last_imp_ch2]}")
        
print(f"time: {time.time()-start}, iteration count: {count}")

Buffer Excluded


,Receiving,VTC2,VMC,Buffer,UMC,SHP,VTC1,HMC,Shipping
Receiving,0,8,0,0,0,0,12,0,0
VTC2,0,0,4,0,0,4,0,4,0
VMC,0,0,0,0,0,0,0,4,5
Buffer,0,0,0,0,0,0,0,0,0
UMC,0,0,0,0,0,0,6,0,0
SHP,0,4,5,0,6,0,0,1,0
VTC1,0,0,0,0,0,12,0,0,6
HMC,0,0,0,0,0,0,0,0,9
Shipping,0,0,0,0,0,0,0,0,0


Total: 1512, changed: VMC and UMC
time: 0.010534048080444336, iteration count: 1
